#### Notebook Description


## Index


In [1]:
import pandas as pd



In [2]:
df = pd.read_csv('./Data/zip_income_3col')
df = df.drop('Unnamed: 0', axis=1)

In [3]:
df.head()

,zipcode,income,state
0,0,115806207,AL
1,35004,290260,AL
2,35005,121690,AL
3,35006,57418,AL
4,35007,697882,AL


### Input Zip Codes

Get list of zip codes to return.  Later add functionality to take in zip codes from CSV.

In [43]:
zips_dict = {'zipcode': {},
             'income': {},
             'state': {}}
zips_df = pd.DataFrame(zips_dict)
print('Enter zip codes for review.  When all are added, type done as input.')
while True:
    new_zip = input('Zip code: ')
    if new_zip.lower() == 'done':
        break
    try:
        zips_df = zips_df.append(df[(df.zipcode==int(new_zip))])
    except:
        print('Invalid zip code entered.')
zips_df = zips_df.reset_index(drop=True)
zips_df

Enter zip codes for review.  When all are added, type done as input.
Zip code: 36005
Zip code: 36007
Zip code: done


,zipcode,income,state
0,36005.0,28829.0,AL
